# Feed calculator

In [1721]:
import pandas as pd
from pulp import *

# Basic Model

## Process the dataset

In [1722]:
pd.set_option('display.max_columns', 500) #To show all the columns when neccessary
df = pd.read_excel("nutrition.xlsx",nrows=64)

In [1723]:
df.columns = ['ingredient' if col.startswith('INGREDIENT') else col for col in df]

In [1724]:
food_items = list(df['ingredient'])

In [1725]:
print("So, the food items to consdier, are\n"+"-"*100)
for f in food_items:
    print(f,end=', ')

So, the food items to consdier, are
----------------------------------------------------------------------------------------------------
barley, blood, boneash, copra, cotton, fats, fish, fishlq, gnseeds, gncake, maize, maizebranhighq, maizebranlowq, mbmeal, sugars, pkcake, palmoil, rapecake, rapemeal, rice, sesamecake, sorghum, soybeanexp, soybeanhulls, soybeanmeal, soybeanheat, sunflower, sunflowerseeds, tapbran, caswhole, casfine, cascoarse, wheat, wheatbran, dicaph, monocapo, lime, shells, salt, 

In [1726]:
import numpy as np
df.shape

(39, 73)

In [1727]:
np.random.seed(0)
# df['Price/kg'] = random.rand(39,1)*10
df['Price/kg'] = np.ones(39)*10

## Create dictionary of nutrition for all food items

In [1728]:
#From here, write the interface of what nutrition dataframe to choose.
#And the format of the data frame should include [Name]
#nutrition_df=???

In [1729]:
nutrition_df=pd.read_excel('quantity.xlsx')

In [1730]:
nutrition_df

,Afkorting in WEBCALCULATOR,unit,Minimum,Maximum,Rule in webapp
0,oebr,kcal,2650.0,2700.0,oebr btw 2650 and 2700
1,oelh,kcal,0.0,NaN,oelh gt 0
2,cp,g/kg,152.0,200.0,cp btw 152 and 200
3,cfat,g/kg,0.0,NaN,cfat gt 0
4,cfibre,g/kg,0.0,65.0,cfibre btw 0 and 65
5,staew,g/kg,310.0,NaN,staew gt 310
6,ca,g/kg,7.9,8.9,ca btw 7.9 and 8.9
7,na,g/kg,1.4,1.7,na btw 1.4 and 1.7
8,opp,g/kg,3.8,NaN,opp gt 3.8
9,dlysp,g/kg,8.2,9.1,dlysp btw 8.2 and 9.1


In [1731]:
#Extract all the nutritions included
nutrition=[]
for i in range (len(nutrition_df)):
    nutrition.append(nutrition_df.iloc[i][0])

In [1732]:
#Create dictionay for every single nutrition. The dictionary includes the amount contained in different ingredient.
for i in range(len(nutrition)):
    vars()[nutrition[i]] = dict(zip(food_items,df[nutrition[i]]))

In [1733]:
food_vars = LpVariable.dicts("Portion",food_items,lowBound=0,cat='Continuous')

## Dealing with costs

In [1734]:
print(food_items)

['barley', 'blood', 'boneash', 'copra', 'cotton', 'fats', 'fish', 'fishlq', 'gnseeds', 'gncake', 'maize', 'maizebranhighq', 'maizebranlowq', 'mbmeal', 'sugars', 'pkcake', 'palmoil', 'rapecake', 'rapemeal', 'rice', 'sesamecake', 'sorghum', 'soybeanexp', 'soybeanhulls', 'soybeanmeal', 'soybeanheat', 'sunflower', 'sunflowerseeds', 'tapbran', 'caswhole', 'casfine', 'cascoarse', 'wheat', 'wheatbran', 'dicaph', 'monocapo', 'lime', 'shells', 'salt']


In [1735]:
cost_df_raw=pd.read_excel('costs.xlsx')

In [1736]:
food_items_df=pd.DataFrame(food_items,columns=['food'])

In [1737]:
costs_df=food_items_df.merge(cost_df_raw,left_on='food',right_on='Ingredients')

In [1738]:
costs_df=costs_df.drop('food',axis=1)

In [1739]:
costs_df.head()

,Ingredients,Price
0,barley,105
1,blood,285
2,boneash,70
3,copra,65
4,cotton,80


In [1740]:
costs={}
for i in range (len(costs_df)):
    costs[costs_df['Ingredients'].iloc[i]]=costs_df['Price'].iloc[i]

In [1741]:
print(costs)
len(costs)

{'barley': 105, 'blood': 285, 'boneash': 70, 'copra': 65, 'cotton': 80, 'fats': 600, 'fish': 250, 'fishlq': 220, 'gnseeds': 235, 'gncake': 200, 'maize': 130, 'maizebranhighq': 75, 'maizebranlowq': 60, 'mbmeal': 135, 'sugars': 85, 'pkcake': 50, 'palmoil': 600, 'rapecake': 125, 'rapemeal': 87, 'rice': 45, 'sesamecake': 97, 'sorghum': 150, 'soybeanexp': 150, 'soybeanhulls': 30, 'soybeanmeal': 150, 'soybeanheat': 250, 'sunflower': 81, 'sunflowerseeds': 195, 'tapbran': 90, 'caswhole': 60, 'casfine': 70, 'cascoarse': 65, 'wheat': 110, 'wheatbran': 70, 'dicaph': 350, 'monocapo': 375, 'lime': 20, 'shells': 45, 'salt': 100}


39

## Add the objective function

In [1742]:
prob = LpProblem("Smallholder Layer Starter Diet",LpMinimize)

In [1743]:
# The objective function is added to 'prob' first
prob += lpSum([costs[i]*food_vars[i] for i in food_items]), "Total Cost of the balanced diet"

## Adding constraints

In [1744]:
for i in range(len(nutrition_df)):
    n = vars()[nutrition_df.iloc[i,0]]
    print('Min',i)
    
    prob += lpSum([n[f]* food_vars[f] for f in food_items]) >= nutrition_df['Minimum'][i]
    if (i in nutrition_df[nutrition_df['Maximum']>0].index):
        print('Max',i)
        prob += lpSum([n[f]* food_vars[f] for f in food_items]) <= nutrition_df['Maximum'][i]
    print('------')

Min 0
Max 0
------
Min 1
------
Min 2
Max 2
------
Min 3
------
Min 4
Max 4
------
Min 5
------
Min 6
Max 6
------
Min 7
Max 7
------
Min 8
------
Min 9
Max 9
------
Min 10
------
Min 11
Max 11
------
Min 12
------
Min 13
------
Min 14
------
Min 15
------


In [1745]:
print(costs)
len(costs)

{'barley': 105, 'blood': 285, 'boneash': 70, 'copra': 65, 'cotton': 80, 'fats': 600, 'fish': 250, 'fishlq': 220, 'gnseeds': 235, 'gncake': 200, 'maize': 130, 'maizebranhighq': 75, 'maizebranlowq': 60, 'mbmeal': 135, 'sugars': 85, 'pkcake': 50, 'palmoil': 600, 'rapecake': 125, 'rapemeal': 87, 'rice': 45, 'sesamecake': 97, 'sorghum': 150, 'soybeanexp': 150, 'soybeanhulls': 30, 'soybeanmeal': 150, 'soybeanheat': 250, 'sunflower': 81, 'sunflowerseeds': 195, 'tapbran': 90, 'caswhole': 60, 'casfine': 70, 'cascoarse': 65, 'wheat': 110, 'wheatbran': 70, 'dicaph': 350, 'monocapo': 375, 'lime': 20, 'shells': 45, 'salt': 100}


39

In [1746]:
# The problem data is written to an .lp file
prob.writeLP("SimpleDietProblem.lp")
# The problem is solved using PuLP's choice of Solver
prob.solve(pulp.PULP_CBC_CMD())

1

In [1747]:
print("Solution"+"-"*100)
for v in prob.variables():
    if v.varValue>0 and v.name[0]=='P':
        print(v.name, "=", v.varValue,'kg')

Solution----------------------------------------------------------------------------------------------------
Portion_boneash = 0.0018651074 kg
Portion_caswhole = 0.073674343 kg
Portion_fish = 0.070430241 kg
Portion_maizebranhighq = 0.13354586 kg
Portion_monocapo = 0.00083770807 kg
Portion_rapemeal = 0.20423689 kg
Portion_rice = 0.48865891 kg
Portion_salt = 0.0012428947 kg
Portion_wheat = 0.067520956 kg


In [1748]:
print("The total cost of this balanced diet is: {} in Nigeria Currency ".format(round(value(prob.objective),2)))

The total cost of this balanced diet is: 79.8 in Nigeria Currency 


# Extension 1-1

In [1749]:
#Interface for selecting the Number of ingredients
MaxNum=3

In [1750]:

prob2 = LpProblem("Smallholder Layer Starter Diet",LpMinimize)

food_chosen = LpVariable.dicts("Chosen",food_items,0,1,cat='Integer')

# The objective function is added to 'prob' first
prob2 += lpSum([costs[i]*food_vars[i] for i in food_items]), "Total Cost of the balanced diet"

for i in range(len(nutrition_df)):
    n = vars()[nutrition_df.iloc[i,0]]
#     print(n)
    print('Min',i)

    prob2 += lpSum([n[f]* food_vars[f] for f in food_items]) >= nutrition_df['Minimum'][i]
    if (i in nutrition_df[nutrition_df['Maximum']>0].index):
        print('Max',i)
        prob2 += lpSum([n[f]* food_vars[f] for f in food_items]) <= nutrition_df['Maximum'][i]
    print('------')

for f in food_items:
#     prob2 += food_vars[f]>= food_chosen[f]*0.1
    prob2 += food_vars[f]>= 0

    prob2 += food_vars[f]<= food_chosen[f]*1e5
prob2+= lpSum([food_chosen[f] for f in food_items])<=MaxNum
# The problem is solved using PuLP's choice of Solver
prob2.solve(pulp.PULP_CBC_CMD())

    
print("Solution"+"-"*100)
for v in prob2.variables():
    if v.varValue>0 and v.name[0]=='P':
        print(v.name, "=", v.varValue,'kg')
print("The total cost of this balanced diet is: {} in Nigeria Currency ".format(round(value(prob2.objective),2)))

Min 0
Max 0
------
Min 1
------
Min 2
Max 2
------
Min 3
------
Min 4
Max 4
------
Min 5
------
Min 6
Max 6
------
Min 7
Max 7
------
Min 8
------
Min 9
Max 9
------
Min 10
------
Min 11
Max 11
------
Min 12
------
Min 13
------
Min 14
------
Min 15
------
Solution----------------------------------------------------------------------------------------------------
Portion_fish = 0.13110074 kg
Portion_wheat = 0.53863388 kg
Portion_wheatbran = 0.42978379 kg
The total cost of this balanced diet is: 122.11 in Nigeria Currency 


# Extension 1-0

In [1751]:
#Interface 

In [1752]:
ingre_percentage_df=pd.read_excel('ingredients_percentage.xlsx')

In [1753]:
ingre_percentage_df.head()

,Afkorting in WEBCALCULATOR,unit,Minimum,Maximum,Rule in webapp
0,barley,%,1,7.0,barley btw 0.01 and 0.07
1,blood,%,1,3.0,blood btw 0.01 and 0.03
2,boneash,%,1,3.0,boneash btw 0.01 and 0.03
3,mbmeal,%,1,4.0,mbmeal btw 0.01 and 0.04
4,cotton,%,1,4.0,cotton btw 0.01 and 0.04


In [1754]:
prob3 = LpProblem("Smallholder Layer Starter Diet",LpMinimize)
prob3 += lpSum([costs[i]*food_vars[i] for i in food_items]), "Total Cost of the balanced diet"

for i in range(len(nutrition_df)):
    n = vars()[nutrition_df.iloc[i,0]]
#     print(n)
#     print('Min',i)
    
    prob3 += lpSum([n[f]* food_vars[f] for f in food_items]) >= nutrition_df['Minimum'][i]
    
    if (i in nutrition_df[nutrition_df['Maximum']>0].index):
#         print('Max',i)
        prob3 += lpSum([n[f]* food_vars[f] for f in food_items]) <= nutrition_df['Maximum'][i]
#     print('------')

for f in food_items:
    prob3 += food_vars[f]>= 0
    prob3 += food_vars[f]<= food_chosen[f]*100

    
# prob3+= lpSum([food_chosen[f] for f in food_items])<=6


for i in range(len(ingre_percentage_df)):
    pmin=ingre_percentage_df.iloc[i][2]/100
    x_i=ingre_percentage_df.iloc[i][0]
    prob3 += -100*(1-food_chosen[x_i])+pmin*lpSum([food_vars[f] for f in food_items])<=food_vars[x_i]
    
    if (i in ingre_percentage_df[ingre_percentage_df['Maximum']>0].index):
#         print('Max',i)
        pmax=ingre_percentage_df.iloc[i][3]/100
        prob3 += pmax*lpSum([food_vars[f] for f in food_items])>=food_vars[x_i]
                                                                          

# The problem is solved using PuLP's choice of Solver
prob3.solve(pulp.PULP_CBC_CMD())
print("Solution"+"-"*100)

for v in prob3.variables():
#     if v.varValue>0 and v.name[0]=='P':
    if v.varValue>0 :
        print(v.name, "=", v.varValue,'kg')
print("The total cost of this balanced diet is: {} in Nigeria Currency ".format(round(value(prob3.objective),2)))

Ingredients=[]
Amount=[]
for v in prob3.variables():
    if v.varValue>0 and v.name[0]=='P':
        Ingredients.append(v.name[8:])
        Amount.append(v.varValue)
total_cost=round(value(prob3.objective),2)
result_df={'Ingredients':Ingredients,'Amount':Amount}
result_df=pd.DataFrame(result_df)
result_df['total_cost']=total_cost

Solution----------------------------------------------------------------------------------------------------
Chosen_barley = 1.0 kg
Chosen_blood = 1.0 kg
Chosen_copra = 1.0 kg
Chosen_fish = 1.0 kg
Chosen_maizebranhighq = 1.0 kg
Chosen_maizebranlowq = 1.0 kg
Chosen_palmoil = 1.0 kg
Chosen_rapecake = 1.0 kg
Chosen_rapemeal = 1.0 kg
Chosen_rice = 1.0 kg
Chosen_sesamecake = 1.0 kg
Chosen_sugars = 1.0 kg
Chosen_tapbran = 1.0 kg
Chosen_wheat = 1.0 kg
Chosen_wheatbran = 1.0 kg
Portion_barley = 0.077685928 kg
Portion_blood = 0.01109799 kg
Portion_copra = 0.050178481 kg
Portion_fish = 0.1109799 kg
Portion_maizebranhighq = 0.22195979 kg
Portion_maizebranlowq = 0.1109799 kg
Portion_palmoil = 0.033649752 kg
Portion_rapecake = 0.044391959 kg
Portion_rapemeal = 0.044391959 kg
Portion_rice = 0.061068869 kg
Portion_sesamecake = 0.028329307 kg
Portion_sugars = 0.01109799 kg
Portion_tapbran = 0.079603353 kg
Portion_wheat = 0.16889385 kg
Portion_wheatbran = 0.055489949 kg
The total cost of this balanced 

In [1755]:
result_df

,Ingredients,Amount,total_cost
0,barley,0.077686,131.3
1,blood,0.011098,131.3
2,copra,0.050178,131.3
3,fish,0.110980,131.3
4,maizebranhighq,0.221960,131.3
5,maizebranlowq,0.110980,131.3
6,palmoil,0.033650,131.3
7,rapecake,0.044392,131.3
8,rapemeal,0.044392,131.3
9,rice,0.061069,131.3
